In [43]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.team_predicter import TeamPredicter, SolverMode
from modules.utils import getDataFilesSorted

import config

In [44]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 1
TOTAL_BUDGET = 1000

In [45]:
current_team_names = {"Jordan Pickford",
                      "Fabian Schär",
                      "Daniel Ballard",
                      "Milos Kerkez",
                      "Rayan Aït-Nouri",
                      "Bryan Mbeumo",
                      "Jacob Murphy",
                      "Brennan Johnson",
                      "Alex Iwobi",
                      "Alexander Isak",
                      "Jarrod Bowen",
                      "André Onana",
                      "Nathan Collins",
                      "Morgan Rogers",
                      "Yoane Wissa"
                      }

In [46]:
filesSorted = getDataFilesSorted()
dataFileName = filesSorted[-1]
print(dataFileName)

{'name': './data/player_stats/26/data_1.csv', 'season': 26, 'gameweek': 1}


In [47]:
with open(scores_filename, "r") as f:
    all_player_data = pd.read_json(f,orient="records")

In [48]:
print(all_player_data)

      id                          name  cost  ict_index  total_points  \
0      0             David Raya Martín    55        4.9            10   
1      1    Kepa Arrizabalaga Revuelta    45        0.0             0   
2      2                     Karl Hein    40        0.0             0   
3      3                 Tommy Setford    40        0.0             0   
4      4  Gabriel dos Santos Magalhães    60        2.7             6   
..   ...                           ...   ...        ...           ...   
694  694                  Leon Chiwome    45        0.0             0   
695  695                 Nathan Fraser    45        0.0             0   
696  696                    Jhon Arias    55        1.6             1   
697  697            David Møller Wolfe    45        1.0             1   
698  698            Jackson Tchatchoua    45        0.0             0   

     points_per_game  form status  starts_per_90 position team  season  \
0                 10    10      a           1.00 

In [49]:
current_team = Team.fromNameSet(current_team_names, all_player_data)

André Onana a
Nathan Collins a
Brennan Johnson a
Fabian Schär a
Alex Iwobi a
Jarrod Bowen a
Alexander Isak n
Jordan Pickford a
Daniel Ballard a
Jacob Murphy a
Bryan Mbeumo a
Rayan Aït-Nouri a
Yoane Wissa d
Milos Kerkez a
Morgan Rogers a


In [50]:
current_team_cost = current_team.getTotalCost()
current_team_cost

983

In [51]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)
data = all_data["data"][SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame(data=data)

In [52]:
selectedTeamNames = selected_team_df["name"].values
selected_team = Team.fromNameSet(selectedTeamNames, all_player_data)

Tyrick Mitchell a
Rayan Aït-Nouri a
Riccardo Calafiori a
Daniel Ballard a
Fabian Schär a
Erling Haaland a
Chris Wood a
Richarlison de Andrade a
Nick Pope a
Dean Henderson a
Brennan Johnson a
Matt O'Riley a
Mohammed Kudus a
Antoine Semenyo a
Georginio Rutter a


In [53]:
display(selected_team)

In [54]:
new_players = selected_team - current_team
print(new_players)


Total Score: 0

Goalkeepers:
- Nick Pope (Captain) 	Score: 0	Cost: 50	Fixture Difficulty: 0.0
- Dean Henderson (Vice Captain) 	Score: 0	Cost: 50	Fixture Difficulty: 0.0
Defenders:
- Tyrick Mitchell	Score: 0	Cost: 50	Fixture Difficulty: 0.0
- Riccardo Calafiori	Score: 0	Cost: 55	Fixture Difficulty: 0.0
Attackers:
- Erling Haaland	Score: 0	Cost: 140	Fixture Difficulty: 0.0
- Chris Wood	Score: 0	Cost: 76	Fixture Difficulty: 0.0
- Richarlison de Andrade	Score: 0	Cost: 66	Fixture Difficulty: 0.0
Midfielders:
- Matt O'Riley	Score: 0	Cost: 55	Fixture Difficulty: 0.0
- Mohammed Kudus	Score: 0	Cost: 65	Fixture Difficulty: 0.0
- Antoine Semenyo	Score: 0	Cost: 71	Fixture Difficulty: 0.0
- Georginio Rutter	Score: 0	Cost: 60	Fixture Difficulty: 0.0


In [55]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -14


In [56]:
HEURISTIC = "combined"
MODE = SolverMode.CHEAPEST_FIRST

In [57]:
all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(24.369743581961693, 24.369743581961693)

In [58]:
START_SAMPLE_GAMEWEEK = config.CURRENT_GAMEWEEK
END_SAMPLE_GAMEWEEK = min(config.CURRENT_GAMEWEEK+2, 39)

In [59]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK, config.CURRENT_SEASON)

new_players.recalculateFixtureDifficulty(matrix)
new_players.calculateScore(HEURISTIC)

selected_team.recalculateFixtureDifficulty(matrix)
selected_team.calculateScore(HEURISTIC)

current_team.recalculateFixtureDifficulty(matrix)
current_team.calculateScore(HEURISTIC)

In [60]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [61]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 1548.647931152011


In [62]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [63]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [64]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [65]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [66]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [67]:
newTeam = getNewTeam(current_team, selected_team, new_players)
newTeam = newTeam.toBenchTeam()
display(newTeam)

Best transfer:
Transfer from Jarrod Bowen -> Richarlison de Andrade:
Old player: Jarrod Bowen	Score: -10.65	Cost: 79	Fixture Difficulty: 14.56886844573797
New player: Richarlison de Andrade	Score: 262.32	Cost: 66	Fixture Difficulty: -11.655094756590374
- Cost change: -13
- Score change: 272.98
Suggested triple captain: Rayan Aït-Nouri
